In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp_orderly.onboarding.faucet import request_testnet_funds as faucet
from emp_orderly.onboarding import deposit
from emp_orderly_types import PerpetualAssetType, Interval, OrderType

load_dotenv()

True

In [10]:
wallet = PrivateKeyWallet(private_key=os.getenv("PRIVATE_KEY"))
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [11]:
print(wallet.address)

0xd3E45819C43e582C7Ecfe2e7BFcD2544a7e53e4c


In [12]:
await faucet(wallet)

True

In [13]:
await deposit(wallet)

'0xb09abcf936f0fab414d7f81755f2e1191796b167424187bb4ba07f9c9e8e8bcd'

In [14]:
print(await wallet.balance())

109956765500000000


In [16]:
class EMSB(Strategy):
    order_size: float = 0.5
    nine_days_length: int = 9
    fourteen_days_length: int = 14
    length: int = 14
    
    @classmethod
    def update_lags(cls, nine_days_length, fourteen_days_length, length):
        cls.nine_days_length = nine_days_length
        cls.fourteen_days_length = fourteen_days_length
        cls.length = length

    def init(self):
        close = self.data.close
        self.ema_nine_days = self.I(EMA, close, self.nine_days_length)
        self.ema_fourteen_days = self.I(EMA, close, self.fourteen_days_length)
        self.slope = self.I(SLOPE, close, self.length)

    def next(self):
        if (
            crossover(self.ema_nine_days, self.ema_fourteen_days) &
            (self.slope > 15)
        ):
            self.position.close()
            self.buy(size=self.order_size)
        elif (
            crossover(self.ema_fourteen_days, self.ema_nine_days) &
            (self.slope < -15)
        ):
            self.position.close()
            self.sell(size=self.order_size)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(EMSB)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.APT,
)

# backtest
tester.backtest()

Start                     2024-09-16 17:15:00
End                       2024-09-21 09:15:00
Duration                      4 days 16:00:00
Exposure Time [%]                         0.0
Equity Final [$]                       1000.0
Equity Peak [$]                        1000.0
Return [%]                                0.0
Buy & Hold Return [%]               24.308715
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [ ]:
# plot
tester.plot(show_price_data=False)
plt.show()